# Web Scraping

If the HTML is well structured and the tables are well formatted we can use pandas.read_html to extract the tables and into dataframes easily from the website.

In [21]:

# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [22]:
# getting the url into a variable

url = 'https://en.wikipedia.org/wiki/World_population'

In [23]:
tables = pd.read_html(url)

In [24]:
# Here we can view the population of each continent and highest populated country and city per continent
continents_population = tables[2]
continents_population

type(continents_population)

pandas.core.frame.DataFrame

Pandas does not read the sidenote links present in the tables leaving us to clean the note holders from the table. Also the table is converted to a Pandas DataFrame automatically.

In [25]:
# Cleaning the df of the note holders
for col in ['Region','Most populous country','Population (millions)']:
  continents_population[col] = continents_population[col].str.replace(r'\[.*?\]','',regex = True)

In [26]:
continents_population

,Region,Density (inhabitants/km2),Population (millions),Most populous country,Most populous city (metropolitan area)
0,Asia,104.1,4641,"1,439,090,595 – India","13,515,000 – Tokyo Metropolis (37,400,000 – Gr..."
1,Africa,44.4,1340,"0,211,401,000 – Nigeria","09,500,000 – Cairo (20,076,000 – Greater Cairo)"
2,Europe,73.4,747,"0,146,171,000 – Russia, approx. 110 million in...","13,200,000 – Moscow (20,004,000 – Moscow metro..."
3,Latin America,24.1,653,"0,214,103,000 – Brazil","12,252,000 – São Paulo City (21,650,000 – São ..."
4,Northern America,14.9,368,"0,332,909,000 – United States","08,804,000 – New York City (23,582,649 – New Y..."
5,Oceania,5,42,"0,025,917,000 – Australia","05,367,000 – Sydney"
6,Antarctica,~0,0.004,N/A,"00,001,258 – McMurdo Station"


In [27]:
# we can also save the table in an excel format
continents_population.to_excel('continents_pop.xlsx',index = False)

## BeautifulSoup
Let's try a harder example using BeautifulSoup. BeautifulSoup gives us more control on how we extract our data than pandas.read_html

In [28]:
url_2 = 'https://www.scrapethissite.com/pages/forms/'
# getting the HTML
html = requests.get(url_2).text

In [29]:
soup = BeautifulSoup(html,'html.parser')

In [30]:
#finding a table
table = soup.find('table',{'class':'table'})
table

<table class="table">
<tr>
<th>
                            Team Name
                        </th>
<th>
                            Year
                        </th>
<th>
                            Wins
                        </th>
<th>
                            Losses
                        </th>
<th>
                            OT Losses
                        </th>
<th>
                            Win %
                        </th>
<th>
                            Goals For (GF)
                        </th>
<th>
                            Goals Against (GA)
                        </th>
<th>
                            + / -
                        </th>
</tr>
<tr class="team">
<td class="name">
                            Boston Bruins
                        </td>
<td class="year">
                            1990
                        </td>
<td class="wins">
                            44
                        </td>
<td class="losses">
                            2

In [31]:
# finding all rows in the data
rows = table.find_all('tr')

hockey_data = []

#finding all the columns and cleaning the data of these html tags
for row in rows:
  cols = row.find_all(['td','th'])
  data = [col.text.strip() for col in cols]
  hockey_data.append(data)

In [32]:
# converting the data into a dataframe
hockey_df = pd.DataFrame(data = hockey_data[1:],columns = hockey_data[0])

In [33]:
#Results!
hockey_df

,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,,0.55,299,264,35
1,Buffalo Sabres,1990,31,30,,0.388,292,278,14
2,Calgary Flames,1990,46,26,,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,,0.425,273,298,-25
5,Edmonton Oilers,1990,37,37,,0.463,272,272,0
6,Hartford Whalers,1990,31,38,,0.388,238,276,-38
7,Los Angeles Kings,1990,46,24,,0.575,340,254,86
8,Minnesota North Stars,1990,27,39,,0.338,256,266,-10
9,Montreal Canadiens,1990,39,30,,0.487,273,249,24


if you go to url_2 you will notice that the above table is incomplete as there are next buttons to view the remaining data. Luckily in this case the pagination is traditional as the next page links to a different url like this : "Next". If clicking "Next" loads new rows via JavaScript, then requests and BeautifulSoup won’t see them (they only get the raw HTML).(We'll look for an example to show this scenario later). For now lets go ahead and read the whole table data from the traditional pagination example we have.

In [34]:
base_url = "https://www.scrapethissite.com/pages/forms/?page_num={}"
all_data = []

# Loop through all pages
for page_num in range(1, 25):
    url = base_url.format(page_num)
    print(f"Fetching page {page_num}...")

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    table = soup.find("table", class_="table")
    rows = table.find_all("tr", class_="team")

    for row in rows:
        cells = row.find_all("td")
        data = [cell.get_text(strip=True) for cell in cells]
        all_data.append(data)

# Create DataFrame
columns = ["Team Name", "Year", "Wins", "Losses", "OT Losses", "Win %", "Goals For", "Goals Against", "+ / -"]
df = pd.DataFrame(all_data, columns=columns)

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Fetching page 16...
Fetching page 17...
Fetching page 18...
Fetching page 19...
Fetching page 20...
Fetching page 21...
Fetching page 22...
Fetching page 23...
Fetching page 24...


In [35]:
df.head()

,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For,Goals Against,+ / -
0,Boston Bruins,1990,44,24,,0.55,299,264,35
1,Buffalo Sabres,1990,31,30,,0.388,292,278,14
2,Calgary Flames,1990,46,26,,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,,0.425,273,298,-25


In [36]:
df.tail()

,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For,Goals Against,+ / -
577,Tampa Bay Lightning,2011,38,36,8,0.463,235,281,-46
578,Toronto Maple Leafs,2011,35,37,10,0.427,231,264,-33
579,Vancouver Canucks,2011,51,22,9,0.622,249,198,51
580,Washington Capitals,2011,42,32,8,0.512,222,230,-8
581,Winnipeg Jets,2011,37,35,10,0.451,225,246,-21


Now we will go ahead and try scraping data from a website where data is loaded dynamically using javascript. We can not use BeautifulSoup in this case because requests loads the HTML of the website before javascript is run. So in this case we will also require Selenium.

In [37]:
# Set up driver 
driver = webdriver.Chrome()  

driver.get("https://www.scrapethissite.com/pages/ajax-javascript/")  
# Click the year 2015
driver.find_element(By.ID, "2015").click()
time.sleep(10)  # wait for data to load

# Grab all rows now visible in the table
rows = driver.find_elements(By.CSS_SELECTOR, "table.table tbody tr")
data = []

for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    data.append([col.text for col in cols])

columns = ["Title", "Nominations", "Awards", "Best Picture"]
df = pd.DataFrame(data, columns=columns)


driver.quit()

In [38]:
df

,Title,Nominations,Awards,Best Picture
0,Spotlight,6,2,
1,Mad Max: Fury Road,10,6,
2,The Revenant,12,3,
3,Bridge of Spies,6,1,
4,The Big Short,5,1,
5,The Danish Girl,4,1,
6,Room,4,1,
7,Ex Machina,2,1,
8,The Hateful Eight,2,1,
9,Inside Out,2,1,


As the 2015 table has worked we will now get tables from all the years present in that webpage

In [39]:
# Set up driver 
driver = webdriver.Chrome()  
driver.get("https://www.scrapethissite.com/pages/ajax-javascript/")
data_dict = {}
for ID in ['2015','2014','2013','2012','2011','2010']:
    # Click the year 2015
    driver.find_element(By.ID,ID).click()
    time.sleep(7)
    rows = driver.find_elements(By.CSS_SELECTOR, "table.table tbody tr")
    data = []
    for row in rows:
        cols = row.find_elements(By.TAG_NAME,'td')
        data.append([col.text.strip() for col in cols])
    data_dict[f'df_{ID}'] = pd.DataFrame(data = data,columns = columns)
    
print('Successfully Compiled all the data!')
driver.quit()
print('Shut down the driver')

Successfully Compiled all the data!
Shut down the driver


### Note
One can also substitute **time.sleep** with

**WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#table-body tr"))
)**

which is more efficient as it:

1. Waits only as long as needed (fast if the data loads quickly).

2. It avoids errors from scraping too early when content hasn't loaded yet

In [40]:
# Having a look at the results
data_dict['df_2013'].head()

,Title,Nominations,Awards,Best Picture
0,12 Years a Slave,9,3,
1,Gravity,10,7,
2,Dallas Buyers Club,6,3,
3,Frozen,2,2,
4,The Great Gatsby,2,2,


In [41]:
data_dict.keys()

dict_keys(['df_2015', 'df_2014', 'df_2013', 'df_2012', 'df_2011', 'df_2010'])

# Amazing work!